In [2]:
import networkx as nx
import osmnx as ox

In [6]:
from mapfunctions.graph_functions import init_graph_bbox
import mapfunctions.constants as const

G_own = init_graph_bbox(const.GRAPH_BBOX_NORTH, const.GRAPH_BBOX_SOUTH,
                    const.GRAPH_BBOX_EAST, const.GRAPH_BBOX_WEST,
                    osm_ways_to_delete=const.osm_ways_to_delete)

G_base = ox.load_graphml('./graph_output/base_graph.graphml')

# Graph from city
# G_malaga = ox.graph_from_place('Málaga, Spain', network_type='drive', simplify=False)

In [9]:
route_no_weight = list(nx.shortest_path(G_base, 2094195159, 4531819296))
print(route_no_weight)

[2094195159, 5625095807, 2614757891, 2094195161, 2874546302, 2094195163, 2094195165, 3152120576, 4943984603, 2094195174, 4943984605, 2094195181, 2094195183, 2094195185, 2009794381, 1912826104, 2009794387, 418335688, 418335689, 2009794352, 418335690, 2009794395, 418335691, 5354846229, 5354846230, 2009607498, 7133351411, 8213606002, 11451800014, 7133351410, 418341443, 3152128570, 9905512591, 5426525032, 2884141237, 3152128566, 2405764878, 3152128567, 418341118, 3152128568, 2884141236, 5356561729, 3152128569, 418341130, 418340966, 418340968, 418341185, 418340969, 418340970, 5427464245, 418341179, 418340971, 418340972, 2009794400, 418340956, 5881622803, 418347345, 5881622801, 5881360865, 418497854, 5881360866, 5881360867, 880392334, 2009794407, 418341786, 4192031101, 4192031103, 11149832524, 418347190, 4531819296]


In [23]:
for e in G_base.edges(data=True):
    e['traffic_level'] = 1
    

route_weight = list(nx.shortest_path(G_base, 2094195159, 4531819296, weight='traffic_level'))
print(route_weight)

TypeError: 'tuple' object does not support item assignment

In [16]:
for node in route:
    print(G_own.nodes[node])

{'y': 36.7256051, 'x': -4.4613246, 'street_count': 2}
{'y': 36.7256048, 'x': -4.4612843, 'street_count': 2}
{'y': 36.7256115, 'x': -4.4612529, 'street_count': 2}
{'y': 36.7256238, 'x': -4.4612243, 'street_count': 3}
{'y': 36.7256412, 'x': -4.4612001, 'street_count': 3}
{'y': 36.7256778, 'x': -4.4611734, 'street_count': 2}
{'y': 36.7257197, 'x': -4.4611659, 'street_count': 2}
{'y': 36.7257609, 'x': -4.4611787, 'street_count': 2}
{'y': 36.7257951, 'x': -4.4612098, 'street_count': 2}
{'y': 36.7258174, 'x': -4.4612548, 'street_count': 2}
{'y': 36.7258245, 'x': -4.4613069, 'street_count': 2}
{'y': 36.7258154, 'x': -4.4613585, 'street_count': 3}
{'y': 36.7257955, 'x': -4.4613967, 'street_count': 3}
{'y': 36.7258002, 'x': -4.4614534, 'street_count': 2}
{'y': 36.7257759, 'x': -4.4617122, 'street_count': 2}
{'y': 36.7257693, 'x': -4.4617819, 'street_count': 2}
{'y': 36.7257237, 'x': -4.4620078, 'street_count': 3}
{'y': 36.7257264, 'x': -4.4620455, 'street_count': 2}
{'y': 36.7257207, 'x': -4.46

KeyError: 9613272650

In [ ]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['TFG']['graphs'].aggregate([
    {
        '$group': {
            '_id': '$filename', 
            'uniqueDocs': {
                '$first': '$$ROOT'
            }
        }
    }, {
        '$replaceRoot': {
            'newRoot': '$uniqueDocs'
        }
    }
])

result = list(result)

import json
# Save the result to a JSON file
with open('output.json', 'w') as file:
    json.dump(result, file, default=str, indent=4)
    

In [3]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
collection = client['TFG']['graphs_clean']
collection.update_many(
    {},
    [
        {
            "$set": {
                "datetime": {
                    "$dateFromString": {
                        "dateString": "$datetime",
                        "format": "%Y-%m-%d %H:%M:%S"
                    }
                }
            }
        }
    ]
)

print("Datetime fields updated to Date type.")

Datetime fields updated to Date type.


In [11]:
import networkx as nx
import osmnx as ox
G_base = ox.load_graphml('./graph_output/base_graph.graphml')

# Traffic Lights nodes
nodes_with_tl = [6673740454, 6673740455, 1039214555, 1039214910, 1648203777, 4192025593, 880392359, 1052330903, 5376511114, 1143799037, 5840213940, 2094195161, 2874546303, 5356669924, 5341770502, 4664242443, 5343394579, 5341746544, 696433012, 5940544411, 5224706524]

# Set traffic lights to True
for node in nodes_with_tl:
    if node in G_base.nodes:
        G_base.nodes[node]['traffic_light'] = True
        
# Save graph
ox.save_graphml(G_base, './graph_output/base_graph.graphml')